# What's all this about?

There are few aspects of modern life that have not been affected by Covid-19. Like many city's, Newcastle City Council introduced a number of changes with the intention of supporting social distancing. Amogst these were a number to the roads in Gosforth (where I live), closing certain roads to through motor traffic, establishing "Low Traffic Neighbourhoods" (LTNs).

A local campaining group "Space for Gosforth", wrote [this blog article](http://spaceforgosforth.com/bollards) which pointed out that these measures are not new and there are already a number of local areas which are closed to through motor traffic, even if they weren't called LTNs origionally.

These changes have their supporters and their detractors. One common accusation is that LTNs favour better off households, leaving poorer households to suffer higher, displaced traffic with the assicated collision and air polution risk. Is there any evidence to support this accusation related to the changes in Gosforth? 



# Area of Interest

Those who familar with Newcastle will know Gosforth includes some of the more affulent areas of the city. However this is not uniform and not everywhere in or around Gosforth is so well healed. 


The Area of Interest in somewhat arbitary, but for now I have limited myself to to the council wards which include the LTNs in the Space for Gosforth blog. Which is roughly the area bounded by, the Town Moor to the south, the A1 to the West, the Race Course to the north and boundary with North Tyneside to the north-east and cuts through parts of Heaton and Jesmond to the south-east. With the exception of the south-east, this generally conicides with some existing boundaries in the geography of the city.

Since we're interested in the relationship of the relative wealth of neibourhoods which are designed to be "low-traffic" or not.



# How are we looking at this?

There is a crude assumption in this analysis:
* That any _benefits_ of LTNs occur for those households that are included in them. 
* That any _disadvantages_ of LTNs occur outside them.
* 
The truth is, no doubt, a little more complex. Others have debated this at length elsewhere.

The core question here is what is the range of wealth of the different households that are:
a) in the newly established LTNs?
b) in the previously established LTNs?
c) not in any LTNs?

# How do we get this information?

This is 

## Who lives inside and outside of the LTNs?

The full information about who lives where and their releitive wealth is not available to a random punter who want to do their own analysis. From privacy point of view, this is a good thing.

In the absence of data on where individuals live, a useful substitue can be the number of households. 

One of the best 

Just interested in residential properties.




## What are the boundaries of the LTNs

It is surprisingly hard to find a 


## How do we assess releitive wealth?



# Setup some useful imports

In [ ]:
!pip install icecream
!pip install requests
!pip install ipyleaflet
!pip install matplotlib
!pip install descartes
# !pip install osmium
!pip install overpy

import shapely
import os
from pathlib import Path
import hashlib
import requests
from icecream import ic
import re
import json
import shutil
from osgeo import ogr
import geopandas
import matplotlib
from ipyleaflet import Map, basemaps, GeoData, LayersControl

# Some helpful functions


In [ ]:
# osm_file = 'tyne-and-wear-latest.osm.pbf'
# osm_download_url = 'https://download.geofabrik.de/europe/great-britain/england/tyne-and-wear-latest.osm.pbf'
# osm_lastest_md5sum_url = 'https://download.geofabrik.de/europe/great-britain/england/tyne-and-wear-latest.osm.pbf.md5'

# Well known indentifers for the two coordinate systems used here.
# Unprojected, WGS1984, Long, Lat
# Prjected, British National Grid
crs_long_lat = "EPSG:4326"
crs_bng = "EPSG:27700"


def _get_md5sum_local_file(f_path):
    hash = hashlib.md5()
    try:
        if os.path.isfile(f_path):
            with open(f_path, "rb") as lf:
                hash.update(lf.read())
    finally:
        return hash.hexdigest()


def _get_lastest_md5sum(md5sum_url):
    response = requests.get(md5sum_url)
    # We are only interested in the 32 char string, not the file name
    match = re.search("^([0-9a-f]{32})[ ]{2}.*$", response.text)
    return match.group(1)


def _download_file(download_url, target_path):
    if not Path.exists(Path(target_path).parent):
        Path.mkdir(Path(target_path).parent, parents=True)

    with open(target_path, mode="wb") as fb:
        response = requests.get(download_url)
        ic(response)
        fb.write(response.content)


def download_latest_file(download_url, target_filename, md5sum_url=None):
    target_path = f"downloads/{target_filename}"
    if md5sum_url:
        if _get_md5sum_local_file(target_path) != _get_lastest_md5sum(md5sum_url):
            ic("Remote files is different to localfile, downloading update.")
            _download_file(download_url, target_path)
    else:
        if not os.path.exists(target_path):
            ic("local file does not exist, downloading new file.")
            _download_file(download_url, target_path)


# https://download.geofabrik.de/europe/great-britain/england/tyne-and-wear.poly

# Download OSM from Geofabrik

In [ ]:
osm_file = "tyne-and-wear-latest.osm.pbf"
osm_download_url = "https://download.geofabrik.de/europe/great-britain/england/tyne-and-wear-latest.osm.pbf"
osm_lastest_md5sum_url = "https://download.geofabrik.de/europe/great-britain/england/tyne-and-wear-latest.osm.pbf.md5"

# download_latest_file(osm_download_url, osm_file, osm_lastest_md5sum_url)

# Setup the routing server

In [ ]:
# !echo {osm_lastest_md5sum_url} osm_lastest_md5sum_url
ic.disable()

car_port = 5001
bike_port = 5002
walk_port = 5003

osrm_docker_params = {
    "driving": ("data/osm/car", car_port, "/opt/car.lua"),
    "cycling": ("data/osm/bike", bike_port, "/opt/bicycle.lua"),
    "walking": ("data/osm/walk", walk_port, "/opt/foot.lua"),
}

_pwd = %pwd

for verb, values in osrm_docker_params.items():
    data_path, host_port, profile_path = values
    ic(data_path, host_port, profile_path, verb)
    fq_path = Path(_pwd).joinpath(data_path)
    ic(fq_path)
    if not Path.exists(fq_path):
        Path.mkdir(fq_path, parents=True)

    shutil.copyfile(f"downloads/{osm_file}", f"{fq_path}/{osm_file}")

    # Prep the data
    !docker run -t -v "{fq_path}:/data" osrm/osrm-backend osrm-extract -p {profile_path} /data/tyne-and-wear-latest.osm.pbf
    !docker run -t -v "{fq_path}:/data" osrm/osrm-backend osrm-partition /data/tyne-and-wear-latest.osrm
    !docker run -t -v "{fq_path}:/data" osrm/osrm-backend osrm-customize /data/tyne-and-wear-latest.osrm

# Download boundaries from Ordnance Survey

In [ ]:
boundaries_download_url = "https://api.os.uk/downloads/v1/products/BoundaryLine/downloads?area=GB&format=GeoPackage&redirect"
boundaries_zip_file = "bdline_gpkg_gb.zip"
boundaries_file = "data/boundaries/data/bdline_gb.gpkg"

download_latest_file(boundaries_download_url, boundaries_zip_file)

if not os.path.exists(boundaries_file):
    shutil.unpack_archive(f"downloads/{boundaries_zip_file}", "data/boundaries")

# Extract just the wards we are interested in

In [ ]:
# Querry the county wards we are interested in and create a new layer
shapely.speedups.disable()
ward_list = [
    "Kenton Ward",
    "Parklands Ward",
    "Dene & South Gosforth Ward",
    "Fawdon & West Gosforth Ward",
    "North Jesmond Ward",
    "Gosforth Ward",
]

# In British National Grid
all_wards_bng = geopandas.read_file(
    Path(_pwd).joinpath(boundaries_file), layer="district_borough_unitary_ward"
)
# ic(all_wards_gdf.length)

all_wards_bng.head()

aoi_wards_bng = all_wards_bng[all_wards_bng["Name"].isin(ward_list)]
# ic(aoi_wards)
aoi_wards_bng = aoi_wards_bng[
    aoi_wards_bng["File_Name"].isin(["NEWCASTLE_UPON_TYNE_DISTRICT_(B)"])
]
# ic(aoi_wards)

# In Lat/Long
aoi_wards_ll = aoi_wards_bng.to_crs(crs="EPSG:4326")

# driver = ogr.GetDriverByName("GPKG")
# dataSource = driver.Open(all_boundaries, 0)
# layer = dataSource.GetLayer('district_borough_unitary_ward')
# ic(layer.GetFeatureCount())

# # layer.SetAttributeFilter(ic("Name ='{}'".format("' or Name ='".join(ward_list))))
# layer.SetAttributeFilter(ic("Name in ('{}')".format("', '".join(ward_list))))

# ic(layer.GetFeatureCount())
# for feature in layer:
#     print(feature.GetField("Name"))

In [ ]:
# ic(_pwd)
aoi_wards_bng.plot()
aoi_wards_ll.plot()

# Download OSM building data (old)

#  Download OSM streets from Overpass

In [ ]:
ic.enable()

overpass_query = """[timeout:25]
[out:xml]
;
(
  way
    ["highway"]
    (54.98785,-1.66885,55.04157,-1.57675);
  relation
    ["highway"]
    (54.98785,-1.66885,55.04157,-1.57675);
);
(
  node(w);
  node(r);
);
out;"""

import overpy
import pandas as pd

api = overpy.Overpass()

# fetch all ways and nodes
result = api.query(overpass_query)
ic(result)
ic(len(result.nodes))
ic(result.nodes[0])

d = {
    "id": [n.id for n in result.nodes],
    "lon": [n.lon for n in result.nodes],
    "lat": [n.lat for n in result.nodes],
}

df = pd.DataFrame(d)
ic(df.head())
ic(df.dtypes)
all_street_nodes = geopandas.GeoDataFrame(
    df.id, crs="EPSG:4326", geometry=geopandas.points_from_xy(df.lon, df.lat)
)
ic(all_street_nodes.head())
ic(all_street_nodes.dtypes)

# all_street_nodes = geopandas.clip(all_street_nodes, aoi_wards_ll)
all_street_nodes.plot()

# Download building from OpenStreetMap via Overpass

In [ ]:
# Download building from OpenStreetMap

# Overpass query
overpass_query = """
[out:xml] [timeout:25];
(
    way["building"]( 54.98785,-1.66885,55.04157,-1.57675);
    relation["building"]( 54.98785,-1.66885,55.04157,-1.57675);
);
(._;>;);
out body;
"""

# response = api.query(overpass_query)
# ic(response)
# ic(len(response.ways))
# ic(len(response.relations))
# ic(response.ways[0])

# use this example https://gis.stackexchange.com/a/328608
import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize, voronoi_diagram
import io
import gdal

# api = overpy.Overpass()

# query = """ YOUR QUERY HERE """
# response = api.query(query)
overpass_url = "http://overpass-api.de/api/interpreter?"

chunk_size = 1024
parameters = {"data": overpass_query}
r = requests.get(overpass_url, stream=True, params=parameters)
# a = None
# for chunk in r.iter_content(chunk_size=chunk_size):
#     if not a:
#         a = chunk
#     pass
#     # fd.write(chunk)
# ic(a)

# geom_types = [
#     "points",
#     "lines",
#     "multilinestrings",
#     "multipolygons",
#     "other_relations",
# ]

xml_path = "data/osm/buildings/overpass_result.xml"
with open(xml_path, "wb") as xml_fb:
    for chunk in r.iter_content(chunk_size=chunk_size):
        xml_fb.write(chunk)

out_file = "data/osm/buildings/osm_buildings.gpkg"

in_ds = ogr.Open(xml_path)

out_driver = ogr.GetDriverByName("GPKG")
gdal.SetConfigOption("OSM_USE_CUSTOM_INDEXING", "NO")
# Delete if previously exists
if os.path.exists(out_file):
    out_driver.DeleteDataSource(out_file)
# Create file and layer
out_ds = out_driver.CreateDataSource(out_file)
in_lyr = in_ds.GetLayerByName("multipolygons")
out_lyr = out_ds.CopyLayer(
    in_lyr, "multipolygons", options=["OVERWRITE=YES", "ENCODING=UTF-8"]
)
out_lyr.SyncToDisk()

In [ ]:
osm_building_gdf = geopandas.read_file(out_file, layer="multipolygons")
ic(len(osm_building_gdf))
ic(osm_building_gdf.columns)
osm_building_gdf[["building", "name"]].columns
(osm_building_gdf[["building", "osm_way_id"]]).groupby(
    ["building"]
).count().sort_values("osm_way_id")

In [ ]:
parking_query = """[out:xml] [timeout:25];
(
    node["amenity"="parking"]( {{bbox}});
    way["amenity"="parking"]( {{bbox}});
    relation["amenity"="parking"]( {{bbox}});
);
(._;>;);
out body;"""

school_query = """[out:xml] [timeout:25];
(
    node["amenity"="kindergarten"]( {{bbox}});
    node["amenity"="childcare"]( {{bbox}});
    node["amenity"="school"]( {{bbox}});
    way["amenity"="kindergarten"]( {{bbox}});
    way["amenity"="childcare"]( {{bbox}});
    way["amenity"="school"]( {{bbox}});
    relation["amenity"="kindergarten"]( {{bbox}});
    relation["amenity"="childcare"]( {{bbox}});
    relation["amenity"="school"]( {{bbox}});
);
(._;>;);
out body;"""

healthcare_query = """[out:xml] [timeout:25];
(
    node["healthcare"]( {{bbox}});
    node["amenity"="doctors"]( {{bbox}});
    way["healthcare"]( {{bbox}});
    way["amenity"="doctors"]( {{bbox}});
    relation["healthcare"]( {{bbox}});
    relation["amenity"="doctors"]( {{bbox}});
);
(._;>;);
out body;"""

offices_query = """[out:xml] [timeout:25];
(
    node["building"="office"]( {{bbox}});
    node["office"]( {{bbox}});
    way["building"="office"]( {{bbox}});
    way["office"]( {{bbox}});
    relation["building"="office"]( {{bbox}});
    relation["office"]( {{bbox}});
);
(._;>;);
out body;"""

leisure_query = """[out:xml] [timeout:25];
(
    node["leisure"]( {{bbox}});
    way["leisure"]( {{bbox}});
    relation["leisure"]( {{bbox}});
);
(._;>;);
out body;"""

# Needs refinement
outdoor_leisure_query = """[out:xml] [timeout:25];
(
    node["natural"]( {{bbox}});
    node["leisure"="park"]( {{bbox}});
    node["leisure"="golf_course"]( {{bbox}});
    node["leisure"="nature_reserve"]["landuse"="allotments"]( {{bbox}});
    way["natural"]( {{bbox}});
    way["leisure"="park"]( {{bbox}});
    way["leisure"="golf_course"]( {{bbox}});
    way["leisure"="nature_reserve"]["landuse"="allotments"]( {{bbox}});
    relation["natural"]( {{bbox}});
    relation["leisure"="park"]( {{bbox}});
    relation["leisure"="golf_course"]( {{bbox}});
    relation["leisure"="nature_reserve"]["landuse"="allotments"]( {{bbox}});
);
(._;>;);
out body;"""

# old code for parsing OSM overpy results

In [ ]:
# df = {
#     'building': [],
#     'building_levels': [],
#     'name': [],
#     'geometry': []
# }

# 'building': 'apartments', 'building:levels': '15', 'name':

# for ii_w,way in enumerate(response.ways):
#     ls_coords = []
#     # ic(ii_w)

#     df['building'].append(way.tags.get('building', None))
#     df['building_levels'].append(way.tags.get('building:levels', None))
#     df['name'].append(way.tags.get('name', None))

#     for node in way.nodes:
#         ls_coords.append((node.lon,node.lat)) # create a list of node coordinates

#     ls = geometry.LineString(ls_coords)
#     df['geometry'].append(ls) # create a LineString from coords

# merged = linemerge([*lss]) # merge LineStrings
# borders = unary_union(merged) # linestrings to a MultiLineString
# polygons = list(polygonize(borders))
# ic(len(polygons))

# osm_building_gdf = geopandas.geodataframe.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
# ic(len(osm_building_gdf))

In [ ]:
# df = pd.DataFrame(d)
# ic(df.head())
# ic(df.dtypes)
# all_street_nodes = geopandas.GeoDataFrame(df.id, crs='EPSG:4326', geometry=geopandas.points_from_xy(df.lon, df.lat))
# ic(all_street_nodes.head())
# ic(all_street_nodes.dtypes)

# all_street_nodes = geopandas.clip(all_street_nodes, aoi_wards_ll)
# all_street_nodes.plot()


# Borowing from here https://stackoverflow.com/a/45772571

# import osmium as osm
# import pandas as pd

# class OSMHandler(osm.SimpleHandler):
#     def __init__(self):
#         osm.SimpleHandler.__init__(self)
#         self.osm_data = []

#     def tag_inventory(self, elem, elem_type):
#         for tag in elem.tags:
#             self.osm_data.append([elem_type,
#                                    elem.id,
#                                    elem.version,
#                                    elem.visible,
#                                    pd.Timestamp(elem.timestamp),
#                                    elem.uid,
#                                    elem.user,
#                                    elem.changeset,
#                                    len(elem.tags),
#                                    tag.k,
#                                    tag.v])

#     def node(self, n):
#         self.tag_inventory(n, "node")

#     def way(self, w):
#         self.tag_inventory(w, "way")

#     def relation(self, r):
#         self.tag_inventory(r, "relation")


# osmhandler = OSMHandler()
# # scan the input file and fills the handler list accordingly
# osmhandler.apply_file(osm_file)

# # transform the list into a pandas DataFrame
# data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
#                  'user', 'chgset', 'ntags', 'tagkey', 'tagvalue']
# df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
# df_osm = tag_genome.sort_values(by=['type', 'id', 'ts'])

# Test osm_building_gdf

In [ ]:
# # polygons[1:10]
# w = response.ways[0]
# ic(w)
# ic(w.tags)
# osm_building_gdf[0:1].plot()
# osm_building_gdf.plot(column='building')
# ic(osm_building_gdf[0:1])
# # response

# Load the LTN boundaries

In [ ]:
ltns_path = "data/ltns/ltns.geojson"
ltns_ll = geopandas.read_file(ltns_path)
ltns_ll.plot(column="Age")

# Load the demographic data

In [ ]:
# imd_download_url = 'https://data.cdrc.ac.uk/system/files/Local_Data_Spaces/Local_Data_Spaces_E08000021.zip'
# 'https://data-communities.opendata.arcgis.com/datasets/indices-of-multiple-deprivation-imd-2019-1/explore?filters=eyJMQURubSI6WyJOZXdjYXN0bGUgdXBvbiBUeW5lIl19&location=54.988609%2C-1.582592%2C12.00'

# For info see https://data-communities.opendata.arcgis.com/datasets/communities::indices-of-multiple-deprivation-imd-2019-1/about
imd_download_url = (
    "https://opendata.arcgis.com/datasets/4ad3e5a10872455eaa67ce4e663d0d01_0.geojson"
)
imd_filename = "imd.geojson"
imd_path = "data/demography/imd.geojson"

download_latest_file(imd_download_url, imd_filename)

if not os.path.exists(imd_path):
    os.makedirs("data/demography", exist_ok=True)
    # .touch(exist_ok=True)
    shutil.copyfile(f"downloads/{imd_filename}", imd_path)
    # shutil.unpack_archive(f'downloads/{imd_zip_file}', 'data/demography')

all_imd_ll = geopandas.read_file(imd_path)
aoi_imd_ll = all_imd_ll[all_imd_ll["LADnm"].isin(["Newcastle upon Tyne"])]

In [ ]:
aoi_imd_ll.plot(column="IMDRank0")
aoi_imd_ll.head()

# Download building info from Ordnance Survey

In [ ]:
# This is updated every 6 weeks, so download a fresh copy
# Scrape Version Date:2021-09 from https://osdatahub.os.uk/downloads/open/OpenUPRN
"https://api.os.uk/downloads/v1/products/OpenUPRN/downloads?area=GB&format=GeoPackage&redirect"

import timeit

import pandas as pd

osopenuprn_path = "osopenuprn_202108_csv/osopenuprn_202107.csv"
aoi_osopenuprn_path = "osopenuprn_202108_csv/aoi_uprn.shp"

df = pd.read_csv(osopenuprn_path)

ic.enable()
ic(len(df))
ic(df.head())
long_min, lat_min, long_max, lat_max = aoi_wards_ll.total_bounds

# Initial crude clip for speed
# clip north/south, then east/west
df = df[(df.LATITUDE > lat_min) & (df.LATITUDE < lat_max)]
ic(len(df))
df = df[(df.LONGITUDE > long_min) & (df.LONGITUDE < long_max)]
ic(len(df))

aoi_uprn_ll_df = geopandas.GeoDataFrame(
    df, crs="EPSG:4326", geometry=geopandas.points_from_xy(df.LONGITUDE, df.LATITUDE)
)
aoi_uprn_df = geopandas.GeoDataFrame(
    df,
    crs="EPSG:27700",
    geometry=geopandas.points_from_xy(df.X_COORDINATE, df.Y_COORDINATE),
)
# ic(len(aoi_uprn_bng_df))
# don't clip
# aoi_uprn_df = geopandas.clip(aoi_uprn_bng_df, aoi_wards_bng)
ic(len(aoi_uprn_df))
aoi_uprn_df.head()

aoi_uprn_df.to_file(aoi_osopenuprn_path)

# Create a test route

In [ ]:
ic.enable()
from time import sleep

from_nodeid = 59642912  # 55.0079437, -1.6151497
to_nodeid = 7019399129  # 55.0042099, -1.6202478

ic(len(all_street_nodes))

ic(all_street_nodes.index)


# ic(all_street_nodes.id.isin([from_nodeid]))

start_node = all_street_nodes[all_street_nodes.id.isin([from_nodeid])].iloc[0]
end_node = all_street_nodes[all_street_nodes.id.isin([to_nodeid])].iloc[0]

route_osrm_params = {
    "alternatives": "false",
    "steps": "false",
    "annotations": "nodes",
    "geometries": "polyline",
    "overview": "false",
    "continue_straight": "default",
}


# params = {
#     'driving': ('data/osm/car', car_port, '/opt/car.lua'),
#     'cycling': ('data/osm/bike', bike_port, '/opt/bicycle.lua'),
#     'walking': ('data/osm/walk', walk_port, '/opt/foot.lua')
# }

three_test_results = {}

for verb, values in osrm_docker_params.items():
    data_path, host_port, profile_path = values
    ic(verb, data_path, host_port, profile_path)
    host_port = 5000
    fq_path = Path(_pwd).joinpath(data_path)
    # Run the server
    container_guid = !docker run -d -t -i -p {host_port}:5000 -v "{fq_path}:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/tyne-and-wear-latest.osrm
    ic(container_guid)
    sleep(5)
    # 'http://127.0.0.1:5000/route/v1/driving/13.388860,52.517037;13.385983,52.496891?steps=true'
    base_osrm_url = f"http://127.0.0.1:{host_port}/route/v1/{verb}/"
    route_osrm_url = f"{base_osrm_url}{start_node.geometry.x},{start_node.geometry.y};{end_node.geometry.x},{end_node.geometry.y}"
    ic(route_osrm_url)

    response = requests.get(route_osrm_url, route_osrm_params)
    # ic(response.content)

    route = json.loads(response.content)
    three_test_results[verb] = route

    route_nodeids = []
    legs = route["routes"][0]["legs"]
    for l in legs:
        # ic(l['annotation']['nodes'])
        route_nodeids.extend(l["annotation"]["nodes"])

    ic(len(route_nodeids))
    # ['legs']
    route_node_df = all_street_nodes[all_street_nodes.id.isin(route_nodeids)]
    ic(route_node_df.head())
    route_node_df.plot()

    !docker stop {container_guid[0]}

In [ ]:
for verb in ["driving", "cycling", "walking"]:
    details = three_test_results[verb]["routes"][0]
    ic(
        verb,
        details["distance"],
        details["duration"],
        details["weight_name"],
        details["weight"],
    )

# three_test_results[verb]['routes'][0]

# Create list of _potential_ route start and end points

In [ ]:
number_of_routes = 10000
min_seperation = 10  # meters
max_seperation = 150  # meters
ic.enable()
route_list_cols = {
    "route_id": [],
    "start_long": [],
    "start_lat": [],
    "start_node_id": [],
    "end_long": [],
    "end_lat": [],
    "end_node_id": [],
    "straight_line_dist": [],
}


def point_sampler(sample_size=number_of_routes):
    sample_points = aoi_uprn_df.sample(2 * sample_size)
    return sample_points.itertuples()


def get_sample_point(point_iter):
    try:
        next_point = point_iter.__next__()
        return next_point, point_iter
    except StopIteration:
        # ic('extending sampler')
        point_iter = point_sampler(int(number_of_routes))
        return get_sample_point(point_iter)


def append_points(route_list_cols, start_point, end_point):
    # route_list_cols['start_long'].append(start_point.geometry.x)
    # route_list_cols['start_lat'].append(start_point.geometry.y)
    route_list_cols["start_long"].append(start_point.LONGITUDE)
    route_list_cols["start_lat"].append(start_point.LATITUDE)
    route_list_cols["start_node_id"].append(None)

    # route_list_cols['end_long'].append(end_point.geometry.x)
    # route_list_cols['end_lat'].append(end_point.geometry.y)
    route_list_cols["end_long"].append(end_point.LONGITUDE)
    route_list_cols["end_lat"].append(end_point.LATITUDE)
    route_list_cols["end_node_id"].append(None)
    route_list_cols["straight_line_dist"].append(dist)
    return route_list_cols


point_iter = point_sampler(number_of_routes)

for idx in range(0, 2 * number_of_routes, 2):
    route_list_cols["route_id"].append(idx)
    #
    start_point, point_iter = get_sample_point(point_iter)
    dist = 0
    try_pair_count = 0

    while not ((dist > min_seperation) and (dist < max_seperation)):
        if try_pair_count == 10:
            start_point, point_iter = get_sample_point(point_iter)
            dist = 0
            try_pair_count = 0

        try_pair_count = try_pair_count + 1
        end_point, point_iter = get_sample_point(point_iter)
        dist = start_point.geometry.distance(end_point.geometry)
        # ic(end_point)
        # ic(idx, dist)

    route_list_cols = append_points(route_list_cols, start_point, end_point)
    # Add the reverse route
    route_list_cols["route_id"].append(idx + 1)
    route_list_cols = append_points(route_list_cols, end_point, start_point)

# for key, values in route_list_cols.items():
#     ic(key, len(values))

route_list_df = pd.DataFrame(route_list_cols)
route_list_df.to_csv("results/route_list.csv", index=False)
# ic(route_list_df.head())
# while sample_points:

In [ ]:
ic.enable()
# ic(route_list_gdf.head())

# route_list_gdf['geometry'] =
#
# ic(route_list_gdf['start_long', 'start_lat', 'end_long', 'end_lat'])

# [geometry.LineString([(r.start_long, r.start_lat),(r.end_long, r.end_lat)]) for r in route_list_gdf]

# DataFrame.apply(func, axis=0, raw=False, result_type=None, args=(), **kwargs)[source]

# route_list_df['geometry'] = None


# There must be a better, set-wise way of achiving this, but I cannot get any of them to work
for i in route_list_df.index.array:
    r = route_list_df.loc[i]
    route_list_df.loc[i, "geometry"] = geometry.LineString(
        [(r.start_long, r.start_lat), (r.end_long, r.end_lat)]
    )

route_list_gdf = geopandas.GeoDataFrame(
    route_list_df, geometry="geometry", crs="EPSG:4326"
)


####
# Various failed attempts to create lines in a set-wise method below
####

# def create_line(r):
#     # ic(r)
#     return geometry.LineString([(r[0],r[1]),(r[2],r[3])])


# temp_line_list = route_list_gdf[['start_long', 'start_lat', 'end_long', 'end_lat']].apply(create_line, axis=1, raw=True)
# temp_line_list.head()
# route_list_gdf['geometry'] = temp_line_list

# route_list_gdf['geometry'] = ([
#         (route_list_gdf['start_long'], route_list_gdf['start_lat']),
#         (route_list_gdf['end_long'], route_list_gdf['end_lat'])
#     ])

# ic(route_list_gdf.head())
# route_list_gdf['geometry'].apply(lambda r: geometry.LineString([(r.start_long, r.start_lat),(r.end_long, r.end_lat)]))
# axis=1,  result_type='expand',
# route_list_gdf['geometry'] = geometry.LineString([
#     (route_list_gdf['start_long'], route_list_gdf['start_lat']),
#     (route_list_gdf['end_long'], route_list_gdf['end_lat'])
#     ])

# route_list_gdf.set_geometry = 'geometry'

# route_list_gdf = geopandas.GeoDataFrame(route_list_df, geometry=geometry.LineString([
#     (route_list_df.start_long, route_list_df.start_lat),
#     (route_list_df.end_long, route_list_df.end_lat)]),
#     crs='EPSG:4326')


route_list_gdf.to_file("results/route_list.shp")
route_list_gdf.head()

# Snap addresses to road network

Note that this ignores the cases where there are different enterences for different transport types (eg parking at large supermarkets), but prevents a more common problem of addresses snapping to different parts of the network, resulting in routes that are not really comparable.

In [ ]:
# Remove duplicats from aoi_uprn_df
# in some cases (eg a block of flats, or PO boxes) there many be multiple points, representing multiple postal delivery points, all with idenical geographic locations.
# ic(aoi_uprn_df.head())
unique_address_points_by_ll = aoi_uprn_df.groupby(
    ["LATITUDE", "LONGITUDE"], as_index=False
).count()
unique_address_points_by_ll.rename(columns={"UPRN": "my_UPRN"}, inplace=True)
unique_address_points_by_ll.set_index("my_UPRN")
ic(unique_address_points_by_ll.head())
# unique_address_points_by_ll.to_file('results/unique_address_points_by_ll.shp')
unique_address_points_by_bng = aoi_uprn_df.groupby(
    ["X_COORDINATE", "Y_COORDINATE"], as_index=False
).count()
unique_address_points_by_bng.rename(columns={"UPRN": "my_UPRN"}, inplace=True)
unique_address_points_by_bng.set_index("my_UPRN")
ic(unique_address_points_by_bng.head())
ic(aoi_uprn_df.head())
# unique_address_points_by_bng.to_file('results/unique_address_points_by_bng.shp')

# ic(len(unique_address_points))

# # using XY
# # ic| len(aoi_uprn_df): 53992
# # ic| len(unique_address_points): 10737

# # using Lat,Long
# # ic| len(aoi_uprn_df): 53992
# # ic| len(unique_address_points): 42916

# unique_address_points.plot()

In [ ]:
# !pip install shapely==1.8rc1
# !pip show shapely
# from shapely.ops import voronoi_diagram
# # address_polys_gdf = aoi_uprn_df.copy()
# # address_polys_gdf['geometry']
# address_polys = voronoi_diagram(aoi_uprn_df['geometry'])
# ic(len(address_polys))

aoi_uprn_df.head()

In [ ]:
# Do snapping using routing server

data_path, host_port, profile_path = osrm_docker_params["driving"]
ic(data_path, host_port, profile_path)

host_port = 5000
fq_path = Path(_pwd).joinpath(data_path)

container_guid = !docker run -d -t -i -p {host_port}:5000 -v "{fq_path}:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/tyne-and-wear-latest.osrm
ic(container_guid)
sleep(5)

# from_nodeid = 59642912  # 55.0079437, -1.6151497
# to_nodeid = 7019399129  # 55.0042099, -1.6202478

# 55.008281099999998
# -1.615038900000000

# 				-1.615252,
# 				55.008258


# # futher north up Hartley terrace
# 55.008973699999999
# -1.615266800000000


# 'http://127.0.0.1:5000/route/v1/driving/13.388860,52.517037;13.385983,52.496891?steps=true'
# http://{server}/nearest/v1/{profile}/{coordinates}.json?number={number}

# 'http://127.0.0.1:5000/nearest/v1/driving/-1.615038900000000,55.008281099999998?number=1&generate_hints=false'
# 'http://127.0.0.1:5000/nearest/v1/driving/-1.615266800000000,55.008973699999999?number=1&generate_hints=false'

# 'http://127.0.0.1:5000/nearest/v1/driving/-1.6123302,55.0102618?number=1&generate_hints=false'

# 55.0102618,-1.6123302


base_snap_osrm_url = f"http://127.0.0.1:{host_port}/nearest/v1/driving/"


nearest_osrm_params = {"number": 1, "generate_hints": "false"}


request_count = 0

address_street_node_cols = {"UPRN": [], "street_node_id": [], "geometry": []}

########
# Using the routing server here, gaurentees that we are snapping to somewhere on the driving road network (not footpaths etc)
########
mybreak = 0
for address in aoi_uprn_df.itertuples():
    mybreak = mybreak + 1
    # if mybreak>4000:
    #     break

    request_count = request_count + 1
    if request_count % 400 == 0:
        sleep(1)

    # get a list of the nestest street nodes

    nearest_osrm_url = f"{base_snap_osrm_url}{address.LONGITUDE},{address.LATITUDE}"
    # ic(nearest_osrm_url)

    response = requests.get(nearest_osrm_url, nearest_osrm_params)
    # ic(response.content)

    nearest = json.loads(response.content)
    nearby_nodes_ids = nearest["waypoints"][0]["nodes"]
    # ic(address.geometry.x)
    # ic(address.geometry.y)

    nearby_nodes_gdf = all_street_nodes[
        all_street_nodes.id.isin(nearby_nodes_ids)
    ].copy(deep=True)

    # typically this happens if there is an address near teh edge of the study area, and the nearest
    # matching street nodes are outside the study areas. This is literally an edge case that can be ignored.
    if len(nearby_nodes_gdf) > 0:
        address_street_node_cols["UPRN"].append(address.UPRN)
        # reproject for comparision with address data
        nearby_nodes_gdf = nearby_nodes_gdf.to_crs(aoi_uprn_df.crs)
        nearby_nodes_gdf["dist"] = nearby_nodes_gdf["geometry"].apply(
            lambda x: address.geometry.distance(x)
        )
        # ic(nearby_nodes_gdf)
        # get the nearest node
        nearest_node = nearby_nodes_gdf.loc[nearby_nodes_gdf["dist"].idxmin()]

        # add the node ID
        address_street_node_cols["street_node_id"].append(nearest_node.id)
        # create a line between the address and the street node
        address_street_node_cols["geometry"].append(
            geometry.LineString([nearest_node.geometry, address.geometry])
        )

        # r = route_list_df.loc[i]
        # route_list_df.loc[i,'geometry'] = geometry.LineString([(r.start_long, r.start_lat),(r.end_long, r.end_lat)])

address_street_node_gdf = geopandas.GeoDataFrame(
    address_street_node_cols, geometry="geometry", crs=aoi_uprn_df.crs
)
address_street_node_gdf.to_file("results/address_street_node_pairs.shp")
# aoi_uprn_df

# Now stop docker
!docker stop {container_guid[0]}

# Create Cardinal points

In [ ]:
address_street_node_clipped_gdf = geopandas.clip(address_street_node_gdf, aoi_wards_bng)
unique_street_node_ids = address_street_node_clipped_gdf["street_node_id"].unique()
ic(len(address_street_node_gdf))
ic(len(address_street_node_clipped_gdf))
ic(len(unique_street_node_ids))
unique_street_node_ids[0:10]

In [ ]:
# unique_street_nodes_gdf = all_street_nodes[all_street_nodes.id.isin(unique_street_node_ids)].copy(deep=True)
# unique_street_nodes_gdf.head()
# nearby_nodes_gdf

In [ ]:
import shapely
# Create GDF of unique street nodes
unique_street_nodes_gdf = all_street_nodes[all_street_nodes.id.isin(unique_street_node_ids)].copy(deep=True)
# Make a copy of teh Long/Lat values for later use with the routing engine
# Then do everything in BNG for for geo calculations
unique_street_nodes_gdf['LONG'] = unique_street_nodes_gdf['geometry'].x
unique_street_nodes_gdf['LAT'] = unique_street_nodes_gdf['geometry'].y
unique_street_nodes_gdf = unique_street_nodes_gdf.to_crs(aoi_uprn_df.crs)

data_path, host_port, profile_path = osrm_docker_params['driving']
ic(data_path, host_port, profile_path)

host_port = 5000
fq_path = Path(_pwd).joinpath(data_path)

container_guid = !docker run -d -t -i -p {host_port}:5000 -v "{fq_path}:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/tyne-and-wear-latest.osrm
ic(container_guid)
sleep(5)

base_snap_osrm_url = f'http://127.0.0.1:{host_port}/nearest/v1/driving/'

nearest_osrm_params = {
    'number': 1,
    'generate_hints': 'false'
}

request_count = 0

address_street_node_cols = {
    'source_node_id': [],
    'geometry': []
}


target_list_cols = {
    'start_node_id': [],
    'geometry': [],
    'intented_offset_dist': [],
    'intented_offset_bearing': []
}

intented_offsets = [50, 250, 1500]
intented_offset_lambdas = []

intented_offset_lambdas.append( ('n', 50, lambda x, y, z=None: (x+0, y+50)) )
intented_offset_lambdas.append( ('e', 50, lambda x, y, z=None: (x+50, y+0)) )
intented_offset_lambdas.append( ('s', 50, lambda x, y, z=None: (x+0, y-50)) )
intented_offset_lambdas.append( ('w', 50, lambda x, y, z=None: (x-50, y+0)) )
intented_offset_lambdas.append( ('n', 250, lambda x, y, z=None: (x+0, y+250)) )
intented_offset_lambdas.append( ('e', 250, lambda x, y, z=None: (x+250, y+0)) )
intented_offset_lambdas.append( ('s', 250, lambda x, y, z=None: (x+0, y-250)) )
intented_offset_lambdas.append( ('w', 250, lambda x, y, z=None: (x-250, y+0)) )
intented_offset_lambdas.append( ('n', 1500, lambda x, y, z=None: (x+0, y+1500)) )
intented_offset_lambdas.append( ('e', 1500, lambda x, y, z=None: (x+1500, y+0)) )
intented_offset_lambdas.append( ('s', 1500, lambda x, y, z=None: (x+0, y-1500)) )
intented_offset_lambdas.append( ('w', 1500, lambda x, y, z=None: (x-1500, y+0)) )

######## 
# Using the routing server here, gaurentees that we are snapping to somewhere on the driving road network (not footpaths etc)
########

# Create GDF of target points in BNG
for street_node in unique_street_nodes_gdf.itertuples():
    for intented_bearing, intented_dist, func in intented_offset_lambdas:
        target_point = shapely.ops.transform(func, street_node.geometry)

        target_list_cols['start_node_id'].append(street_node.id)
        target_list_cols['geometry'].append(target_point)
        target_list_cols['intented_offset_dist'].append(intented_dist)
        target_list_cols['intented_offset_bearing'].append(intented_bearing)

target_gdf = geopandas.GeoDataFrame(target_list_cols, geometry='geometry', crs=unique_street_nodes_gdf.crs)
target_gdf = target_gdf.to_crs(crs='EPSG:4326')
target_gdf.to_file('results/target_cardial_points.shp')

ic(len(unique_street_nodes_gdf))
ic(len(target_gdf))
ic(len(intented_offset_lambdas))

route_list_cols = {
    'route_id': [],
    'start_long': [],
    'start_lat': [],
    'start_node_id': [],
    'end_long': [],
    'end_lat': [],
    'end_node_id': [],
    'intented_offset_dist': [],
    'intented_offset_bearing': [],
    'geometry': []
}

mybreak = 0
request_count = 0
for target_point in target_gdf.itertuples():
    # mybreak = mybreak +1
    # if mybreak>20:
    #     break

    request_count = request_count + 1
    if request_count % 400 == 0:
        sleep(1)

    # get a list of the nestest street nodes
    nearest_osrm_url = f'{base_snap_osrm_url}{target_point.geometry.x},{target_point.geometry.y}'
    response = requests.get(nearest_osrm_url, nearest_osrm_params)
    nearest = json.loads(response.content)
    nearby_nodes_ids = nearest['waypoints'][0]['nodes']

    # ic(nearby_nodes_ids)

    # both = [7736056868, 725625685]
    # exclude = 725625685

    # Prevent snapping back to the start point
    nearby_nodes_ids = [n_id for n_id in nearby_nodes_ids if not n_id == target_point.start_node_id]

    nearby_nodes_gdf = all_street_nodes[all_street_nodes.id.isin(nearby_nodes_ids)].copy(deep=True)

    # typically this happens if there is an address near teh edge of the study area, and the nearest
    # matching street nodes are outside the study areas. This is literally an edge case that can be ignored.
    if len(nearby_nodes_gdf) > 0:
        # reproject for comparision with address data
        nearby_nodes_gdf = nearby_nodes_gdf.to_crs(aoi_uprn_df.crs)
        nearby_nodes_gdf['dist'] = nearby_nodes_gdf['geometry'].apply(lambda x: target_point.geometry.distance(x))
        # get the nearest node
        nearby_nodes_gdf.to_crs(crs='EPSG:4326')
        nearest_node = nearby_nodes_gdf.loc[nearby_nodes_gdf['dist'].idxmin()]

        # this Should be a query
        # start_node_ll = all_street_nodes[all_street_nodes.id.equals(target_point.start_node_id)]
        start_node = unique_street_nodes_gdf[unique_street_nodes_gdf.id.isin([target_point.start_node_id])].iloc[0]


        # add the node ID
        route_list_cols['route_id'].append(request_count)
        route_list_cols['start_long'].append(start_node.LONG)
        route_list_cols['start_lat'].append(start_node.LAT)
        route_list_cols['start_node_id'].append(target_point.start_node_id)
        route_list_cols['end_long'].append(nearest_node.geometry.x)
        route_list_cols['end_lat'].append(nearest_node.geometry.y)
        route_list_cols['end_node_id'].append(nearest_node.id)
        route_list_cols['intented_offset_dist'].append(target_point.intented_offset_dist)
        route_list_cols['intented_offset_bearing'].append(target_point.intented_offset_bearing)
        # real_dist = start_node_bng.geometry.distance(nearest_node.geometry)
        # ic(target_point.intented_offset_dist, real_dist)
        # route_list_cols['actual_line_dist'].append(real_dist)
        # create a line between the start and actual end point
        geom = geometry.LineString([
            (start_node.geometry.x,start_node.geometry.y),
            (nearest_node.geometry.x, nearest_node.geometry.y)
        ])
        route_list_cols['geometry'].append(geom)

        # r = route_list_df.loc[i]
        # route_list_df.loc[i,'geometry'] = geometry.LineString([(r.start_long, r.start_lat),(r.end_long, r.end_lat)])

route_list_gdf = geopandas.GeoDataFrame(route_list_cols, geometry='geometry', crs=aoi_uprn_df.crs)
# route_list_gdf.plot()
route_list_gdf.to_file('results/route_list_carinal_pairs.shp')
# aoi_uprn_df

# Now stop docker
!docker stop {container_guid[0]}



In [ ]:
# route_list_gdf.head(20)
# ic(nearby_nodes_gdf)
# ic(target_point)
# ic(nearby_nodes_ids)

# both = [7736056868, 725625685]
# exclude = 725625685

# a = [x for x in both if not x == exclude]
# ic(a)
# nearby_nodes_ids
# ic(unique_street_nodes_gdf.head())
# 11143086
# start_node_ll = unique_street_nodes_gdf
# a = unique_street_nodes_gdf[unique_street_nodes_gdf.id.isin([11143086])]
# ic(type(a.iloc[0].geometry))
# # ic(len(a.iloc[0].geometry))
# ic(a.iloc[0])
# ic(type(nearest_node.geometry.x))
# ic(type(target_point.geometry.x))
# ic(type(start_node.geometry.x))
#  ic(nearest_node)
# ic(len(address_street_node_gdf))

# address_street_node_gdf['street_node_id'].unique()

# ic(len(address_street_node_gdf['street_node_id'].unique()))

In [ ]:
#
# "D:\andy\space-for-gosforth\results\demo_route_pairs.shp"
route_list_gdf = geopandas.read_file("results/demo_route_pairs.shp")
route_list_cols = {
    "route_id": [],
    "start_long": [],
    "start_lat": [],
    "start_node_id": [],
    "end_long": [],
    "end_lat": [],
    "end_node_id": [],
    "intented_offset_dist": [],
    "intented_offset_bearing": [],
    "geometry": [],
}

ic(route_list_gdf.columns)
route_list_gdf.columns = route_list_cols.keys()
ic(route_list_gdf.columns)

#  ['route_id', 'start_long', 'start_lat', 'start_node', 'end_long',
#                                    'end_lat', 'end_node_id', 'intented_o', 'intented_1', 'geometry']
ic(route_list_gdf.head())
ic(all_street_nodes.head())
####
# Convert route_list_gdf back to lat/long
route_list_gdf = route_list_gdf.to_crs(crs="EPSG:4326")
# a = route_list_gdf.head()

# DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False,
# right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)[source]


########################
########################
########################
#
#
#  DONT RE-RUN
#
#
########################
########################
########################
# route_list_gdf_old = route_list_gdf

route_list_gdf = route_list_gdf.merge(
    all_street_nodes,
    how="inner",
    left_on="end_node_id",
    right_on="id",
    suffixes=("", "all"),
    copy=True,
)
route_list_gdf["end_long"] = route_list_gdf.apply(
    lambda row: row["geometryall"].x, axis=1
)
route_list_gdf["end_lat"] = route_list_gdf.apply(
    lambda row: row["geometryall"].y, axis=1
)

# .apply(lambda row: row['driving_count'] - row['cycling_count'], axis=1)

# b = a.join(all_street_nodes, on='id', how='inner', lsuffix='route', rsuffix='all')
# ic(a)
# ic(b)
ic(route_list_gdf.head())
# ic(all_street_nodes.head())

In [ ]:
# ic(route_list_gdf.head())
ic(route_list_gdf.columns)
route_list_gdf.drop(columns=["id", "geometryall", "idall", "geometryall"])
ic(route_list_gdf.columns)

ic(len(route_list_gdf))

# Bulk calculate routes

In [ ]:

ic.enable()
route_osrm_params = {
    'alternatives': 'false',
    'steps': 'false',
    'annotations': 'true',
    'geometries': 'geojson',
    'overview': 'full',
    'continue_straight': 'default'
}

all_results = {}

all_results_summary = {
    'route_id': [],
    'verb': [],
    'distance': [],
    'duration': [],
    'weight_name': [],
    'weight': [],
    'geometry': []
}


# params = {
#     'driving': ('data/osm/car', car_port, '/opt/car.lua'),
#     'cycling': ('data/osm/bike', bike_port, '/opt/bicycle.lua'),
#     'walking': ('data/osm/walk', walk_port, '/opt/foot.lua')
# }

for verb, values in osrm_docker_params.items():
    data_path, host_port, profile_path = values
    ic(verb, data_path, host_port, profile_path)
    host_port = 5000
    fq_path = Path(_pwd).joinpath(data_path)
    # Run the server
    container_guid = !docker run -d -t -i -p {host_port}:5000 -v "{fq_path}:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/tyne-and-wear-latest.osrm
    ic(container_guid)
    sleep(5)
    # 'http://127.0.0.1:5000/route/v1/driving/13.388860,52.517037;13.385983,52.496891?steps=true'
    base_osrm_url = f'http://127.0.0.1:{host_port}/route/v1/{verb}/'

    result_list_cols = {
        'route_id': [],
        'node_id': [],
    }

    # ic.disable()
    for route_pairs in route_list_gdf.itertuples():
        # Pause every 100's route, to prevent overloading the routing server    
        if route_pairs.route_id % 200 == 0:
            sleep(1)

        if route_pairs.route_id % 1000 == 0:
            ic(route_pairs.route_id)

        # Pandas(Index=0, route_id=0, start_long=-1.6400524, start_lat=54.9964598, start_node_id=None, end_long=-1.5980157, end_lat=55.0031938, end_node_id=None, straight_line_50=2791.5803767758507)
        route_osrm_url = f'{base_osrm_url}{route_pairs.start_long},{route_pairs.start_lat};{route_pairs.end_long},{route_pairs.end_lat}'
        # reverse the route
        # route_osrm_url = f'{base_osrm_url}{route_pairs.end_long},{route_pairs.end_lat};{route_pairs.start_long},{route_pairs.start_lat}'
        # ic(route_osrm_url)


        # ic(route_pairs.route_id)
        response = requests.get(route_osrm_url, route_osrm_params)
        # ic(response.content)
        route = json.loads(response.content)

        if route['code'] != 'Ok':
            all_results_summary['route_id'].append(route_pairs.route_id)
            all_results_summary['verb'].append(verb)
            all_results_summary['distance'].append(0)
            all_results_summary['duration'].append(0)
            all_results_summary['weight_name'].append('NoRoute')
            all_results_summary['weight'].append(0)
            all_results_summary['geometry'].append(None)
        else:
            details = route['routes'][0]
            all_results_summary['route_id'].append(route_pairs.route_id)
            all_results_summary['verb'].append(verb)
            all_results_summary['distance'].append(details['distance'])
            all_results_summary['duration'].append(details['duration'])
            all_results_summary['weight_name'].append(details['weight_name'])
            all_results_summary['weight'].append(details['weight'])

            route_nodeids = []
            legs = details['legs']
            for l in legs:
                # ic(l['annotation']['nodes'])
                route_nodeids.extend(l['annotation']['nodes'])

            # ic(route_pairs.route_id, len(route_nodeids), route_pairs.straight_line_dist)
            # repeat the route id for all values
            result_list_cols['route_id'].extend([route_pairs.route_id for x in route_nodeids])
            result_list_cols['node_id'].extend(route_nodeids)

            details['geometry']

            new_line = shapely.geometry.LineString(details['geometry']['coordinates'])
            all_results_summary['geometry'].append(new_line)


    # 'results/all_route_node_ids'
    
    ic.enable()
    all_results[verb] = pd.DataFrame(result_list_cols)
    ic(verb, all_results[verb].head())
    all_results[verb].to_csv(f'results/demo/{verb}.csv', index=False)

    !docker stop {container_guid[0]}

# all_results_summary_df = pd.DataFrame(all_results_summary)
# all_results_summary_df.to_csv('results/demo/all_routes_carinal_pairs_summary.csv')
all_results_summary_gdf = geopandas.GeoDataFrame(all_results_summary, geometry='geometry', crs="EPSG:4326")
# all_results_summary_df.to_csv('results/demo/all_routes_carinal_pairs_summary.csv')
all_results_summary_gdf.to_file('results/demo/all_routes_carinal_pairs.shp')


In [ ]:
import json

ic(type(route))
# r_dict = json.loads(route)
j_str = json.dumps(route)  # , 'results/demo/route.json')
with open("results/demo/route.json", "w") as fb:
    fb.write(j_str)

# route_nodes = all_street_nodes[all_street_nodes.id.isin(route_nodeids)]
# ic(route_nodes.head())
# new_line = shapely.geometry.LineString(route_nodes.geometry.to_list())
# ic(new_line)

In [ ]:
#################################
#
# Filter out cases where the failed routes (ie where the routing engine failed to provide a route)
# By inspection there are 27 routes which failed - either returning non-result, or returing a zero-length route
# for all 27 cases it failed in some manner for driving, cycling and walking.
# In each case the route started or ended at one of (a) a dual carrage way slip road or (b) somewhere within a private car park.
#
#################################

all_results_summary_df = pd.read_csv(
    "results/all_routes_carinal_pairs_summary.csv", index_col=0
)
ic(len(all_results_summary_df))
all_results_reverse_summary_df = pd.read_csv(
    "results/all_routes_carinal_pairs_reverse_summary.csv", index_col=0
)
all_results_summary_df = all_results_summary_df.append(all_results_reverse_summary_df)
ic(len(all_results_summary_df))
ic(all_results_summary_df["weight_name"].unique())

# Remove both directions for any routes that failed (even if they only failed in one direction)
failed_routes = all_results_summary_df[
    all_results_summary_df["weight_name"].eq("NoRoute")
]["route_id"]
ic(len(failed_routes))
failed_routes = failed_routes.append(-failed_routes)
ic(len(failed_routes))
ic(failed_routes)
all_results_summary_df = all_results_summary_df[
    all_results_summary_df.route_id.isin(failed_routes) == False
]
ic(len(all_results_summary_df))

filtered_results_summary_df = all_results_summary_df[
    all_results_summary_df.distance > 0
]
ic(len(filtered_results_summary_df))
ic(filtered_results_summary_df["weight_name"].unique())
all_results_summary_df.head(2)
# non_reversible_routes = all_results_summary_df.route_id[all_results_summary_df.route_id.isin(-all_results_summary_df.route_id)]
# # self_join = all_results_summary_df.merge(all_results_summary_df, right_on='')

# ic(all_results_summary_df.route_id.head())
# ic(-all_results_summary_df.route_id.head())

# ic(all_results_summary_df.head())
# ic(all_results_reverse_summary_df.head())
# filtered_results_summary_df = all_results_summary_df[all_results_summary_df.distance > 0]
# ic(filtered_results_summary_df['weight_name'].unique())
# filtered_results_summary_df = all_results_summary_df[all_results_summary_df != all_results_summary_df.start_node_id]

In [ ]:
# Add reverse routes to the route_list_gdf
# route_list_gdf['direction'] = None
ic(len(route_list_gdf))
route_list_gdf.head()
temp = route_list_gdf.copy(deep=True)
route_list_gdf["direction"] = "forward"
temp["direction"] = "reverse"
temp["route_id"] = -temp["route_id"]
ic(len(temp))

route_list_gdf = route_list_gdf.append(temp)
ic(len(route_list_gdf))
# ic(route_list_gdf['direction'].unique())
# ic(temp.head())

In [ ]:
# filtered_results_summary_df.head()

# Pivot the table to compare different transport type side-by-side
piv1 = filtered_results_summary_df.pivot(
    index="route_id",
    columns="verb",
    values=["distance", "duration", "weight_name", "weight"],
)
# Rename the columns (flatten the tuple base columns names that pivot generates)
piv1.columns = [f"{s1}_{s2}" for (s1, s2) in piv1.columns.tolist()]
ic(piv1.columns)

# Join the origional route planning data
piv1["route_bi_id"] = abs(piv1.index)
ic(piv1.head(1))
# left_on='route_bi_id', right_on='route_id')
piv = piv1.merge(route_list_gdf, how="inner", on="route_id")
piv = geopandas.GeoDataFrame(piv, geometry="geometry", crs=crs_long_lat)
piv = piv.to_crs(crs_bng)
#'EPSG:27700'
# Exclude any cases where start and end node are within a meter or two

# Now calculate the ratios in length of the different transport types
# Note: The "s" is "s_length" == the straight line distance. Added this to its own column for convenience
piv["s_length"] = piv.apply(lambda row: row["geometry"].length, axis=1)
piv["d_over_c_length"] = (1.0 * piv.distance_driving / piv.distance_cycling).astype(
    "float64"
)
piv["d_over_w_length"] = (1.0 * piv.distance_driving / piv.distance_walking).astype(
    "float64"
)
piv["c_over_w_length"] = (1.0 * piv.distance_cycling / piv.distance_walking).astype(
    "float64"
)

piv["d_over_s_length"] = (1.0 * piv.distance_driving / piv.s_length).astype("float64")
piv["c_over_s_length"] = (1.0 * piv.distance_cycling / piv.s_length).astype("float64")
piv["w_over_s_length"] = (1.0 * piv.distance_walking / piv.s_length).astype("float64")


# piv[['d_over_c_length', 'd_over_w_length']].head()
# piv_short = piv[['d_over_c_length', 'd_over_w_length']]
# piv_short.info()
# ic(piv.sort_values('d_over_c_length').head())
# interesting_routes = piv[piv.d_over_c_length > 1.2]
# ic(len(interesting_routes))
# all_results_summary_df.head()
# interesting_routes.head()
# route_list_gdf.head(1)
# piv.info
# ic(max(piv.d_over_c_length))
# ic(min(piv.d_over_c_length))
# DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False,
# right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)[source]

# # route_list_gdf.head(1)
# piv.head()
# a = piv.columns
# piv[:,([x == ('distance', 'driving') for x in a])]
# b = piv.get(('distance', 'driving'))
# a.head(1)
# b = piv.loc[:,[('distance', 'driving')]].iloc[0]
# type(b)
# b.head(1)
# pd.show_versions()
piv_short = piv[
    [
        "route_id",
        "route_bi_id",
        "s_length",
        "d_over_c_length",
        "d_over_w_length",
        "c_over_w_length",
        "d_over_s_length",
        "c_over_s_length",
        "w_over_s_length",
    ]
].copy()

# piv_short = piv[[
#     'route_id'
# ]].copy()

# ic(piv_short.describe())
piv[["route_id", "route_bi_id"]].hist()
# route_list_gdf[['route_id']].hist()
# route_list_gdf.head(2)
# ic(piv.crs)


# piv['d_over_c_length'].hist()

In [ ]:
# Check that there where exactly two routes (ie exclude cases where only one of the forward or reverse route was generated)
piv_count = (
    piv[["route_bi_id", "d_over_w_length"]]
    .groupby("route_bi_id", as_index=False)
    .count()
)
# ic(len(piv_count))
ic(piv_count.head(2))
failed_route_ids = piv_count[piv_count["d_over_w_length"].eq(2) == False]
ic(failed_route_ids.head(2))
# failed_route_ids = failed_route_ids['route_bi_id']
# ic(failed_route_ids)
# ic(len(piv_short))
# ic((piv_count[piv_count['d_over_w_length'].eq(2)==False]))
piv = piv[piv["route_bi_id"].isin(failed_route_ids["route_bi_id"]) == False]
# ic(len(piv_short), len(piv_short2), len(piv_short)-len(piv_short2))

In [ ]:
###########
#
# Find the min values of each ratio, between forward and reverse route direction

# piv_bi = piv_short[['route_bi_id','d_over_w_length']].groupby('route_bi_id').idxmin(skipna=False)

# a = piv_short[piv_short['route_bi_id'].eq(67544)]
piv_bi = (
    piv[["route_bi_id", "d_over_w_length"]].groupby("route_bi_id").idxmin(skipna=False)
)
piv_short3 = piv.loc[piv_bi["d_over_w_length"].to_list()]
ic(len(piv), len(piv_short3), len(piv_short) - len(piv_short3))

# ic(len(piv_bi))
# ic(piv_bi.columns)
# ic(piv_bi.head(2))
# ic(len(piv_bi[piv_bi['d_over_w_length'].isna()]))
# ic(piv_short.index.dtype)
# ic(piv_bi['d_over_w_length'].dtype)
# piv_bi_dna = piv_bi['d_over_w_length'].dropna().astype('int32')
# ic(piv_bi_dna.head(2))
# ic(len(piv_bi_dna))
# ic(len(piv_bi_dna[piv_bi_dna.isna()]))

# piv_short3 = piv_short[:,piv_bi_dna['route_bi_id']]
# ic(len(piv_short))
# ic(len(piv_short3))

# piv_short [:,piv_bi['d_over_w_length']

# piv_short2 = piv_short[:,piv_bi['d_over_w_length'].dropna().astype('int32')]
# ic(len(piv_short), len(piv_short2), len(piv_short)-len(piv_short2))

# ic(piv_short2.head(2))

# piv_bi2 = piv_short.groupby('route_bi_id').idxmax()
# # ic(piv_bi2['d_over_w_length'].head(2))
# ic(piv_bi2.head(2))
# ic(piv.index)

# Join back into route_list_gdf
# piv_bi = piv_bi.merge(route_list_gdf, how='inner', on='route_id')
# piv_bi.head()
# piv_bi['route_id'].hist()
# piv_short.head(6)

In [ ]:
piv_short3.head(2)
quan_90 = (
    piv_short3[["intented_offset_dist", "d_over_w_length"]]
    .groupby("intented_offset_dist")
    .quantile(0.9)
)
quan_90.head()

In [ ]:
# ic(piv_short.describe())
# ic(piv_short.head())

# diverge_routes = piv[piv.d_over_w_length > 2.087483]
# diverge_routes_50_ids = diverge_routes[diverge_routes.intented_offset_dist == 50]
# diverge_routes_250_ids = diverge_routes[diverge_routes.intented_offset_dist == 250]
# diverge_routes_1500_ids = diverge_routes[diverge_routes.intented_offset_dist == 1500]


diverge_routes_50_ids = piv[
    (piv.d_over_w_length > 1.348924) & (piv.intented_offset_dist == 50)
]
diverge_routes_250_ids = piv[
    (piv.d_over_w_length > 2.886017) & (piv.intented_offset_dist == 250)
]
diverge_routes_1500_ids = piv[
    (piv.d_over_w_length > 1.862107) & (piv.intented_offset_dist == 1500)
]

# for div_routes in [diverge_routes_50_ids, diverge_routes_250_ids, diverge_routes_1500_ids]:
#     ic(div_routes[['route_id', 'd_over_w_length']].describe())

ic(diverge_routes_50_ids.head())

walking_all_nodes = pd.read_csv(f"results/all_route_list_carinal_pairs_ids/walking.csv")

hw50 = walking_all_nodes[
    walking_all_nodes["route_id"].isin(diverge_routes_50_ids.route_id)
]
hw50 = hw50.groupby("node_id", as_index=True).count()
hw50_gdf = all_street_nodes.merge(
    hw50, how="inner", left_on="id", right_on="node_id", copy=True
)
hw50_gdf.to_file("results/hw50.shp")

hw250 = walking_all_nodes[
    walking_all_nodes["route_id"].isin(diverge_routes_250_ids.route_id)
]
hw250 = hw250.groupby("node_id", as_index=True).count()
hw250_gdf = all_street_nodes.merge(
    hw250, how="inner", left_on="id", right_on="node_id", copy=True
)
hw250_gdf.to_file("results/hw250.shp")

hw1500 = walking_all_nodes[
    walking_all_nodes["route_id"].isin(diverge_routes_1500_ids.route_id)
]
hw1500 = hw1500.groupby("node_id", as_index=True).count()
hw1500_gdf = all_street_nodes.merge(
    hw1500, how="inner", left_on="id", right_on="node_id", copy=True
)
hw1500_gdf.to_file("results/hw1500.shp")

# highlighted_walking_nodes = hw50.join(hw250, lsuffix='_50', rsuffix='_250').join(hw1500, rsuffix='_1500')

# ic(len(highlighted_walking_nodes))


# ic(len(highlighted_walking_nodes))
# ic(highlighted_walking_nodes.columns)
# ic(all_street_nodes.head())
# # highlighted_walking_gdf = all_street_nodes[ all_street_nodes['id'].isin(highlighted_walking_nodes['node_id']) ]

# highlighted_walking_gdf = all_street_nodes.merge(highlighted_walking_nodes, how='inner', left_on='id', right_on='node_id', copy=True)

# highlighted_walking_gdf.to_file('results/highlighted_walking.shp')
# # single_gdf = all_street_nodes.join(single_gdf, how='inner', lsuffix='all', rsuffix=verb)


# b = ['a','b','c']
# # filter out secound geometry column before exporting
# a = a[[col for col in a.columns if col != 'geometryall']]
# # ic(len(a))
# a.head(1)

# a.to_file('results/d_over_w_length_90.shp')
# # piv_short['d_over_c_length'].hist()

# Link back to address data

In [ ]:
address_street_node_gdf = geopandas.read_file("results/address_street_node_pairs.shp")
address_street_node_gdf.columns = ["UPRN", "street_node", "geometry"]
address_polys_gdf = geopandas.read_file("osopenuprn_202108_csv/address_polys.shp")
# hw50_gdf
# hw250_gdf
lookup_df = address_street_node_gdf[["UPRN", "street_node"]].copy()

# ic(hw1500_gdf.head(2))
# ic(lookup_df.head(2))
# ic(aoi_uprn_df.head(2))
# ic(address_polys_gdf.head(2))
# ic(address_street_node_gdf.head(2))

hw_gdfs = {"50m": hw50_gdf, "250m": hw250_gdf, "1500m": hw1500_gdf}

for label, hw_gdf in hw_gdfs.items():
    temp = lookup_df.merge(
        hw_gdf, left_on="street_node", right_on="id", how="inner", copy=True
    )
    ltn_points_df = aoi_uprn_df.merge(
        temp, on="UPRN", how="inner", copy=True, suffixes=("", "_DROP_ME")
    )
    # ic(ltn_points_df.head(2))
    ltn_points_df = ltn_points_df[
        [col for col in ltn_points_df.columns if not "_DROP_ME" in col]
    ]
    # ic(ltn_points_df.head(2))
    ltn_points_gdf = geopandas.GeoDataFrame(
        ltn_points_df, geometry="geometry", crs=aoi_uprn_df.crs
    )
    ltn_points_gdf.to_file(f"results/derived_ltn_point_{label}.shp")

    ltn_polys_df = address_polys_gdf.merge(
        temp, on="UPRN", how="inner", copy=True, suffixes=("", "_DROP_ME")
    )
    ltn_polys_df = ltn_polys_df[
        [col for col in ltn_polys_df.columns if not "_DROP_ME" in col]
    ]
    ltn_polys_gdf = geopandas.GeoDataFrame(
        ltn_polys_df, geometry="geometry", crs=address_polys_gdf.crs
    )
    ltn_polys_gdf.to_file(f"results/derived_ltn_poly_{label}.shp")

# Calculate difference between different travel styles

In [ ]:
# ic(all_street_nodes.head())
ic.disable()
single_df = None

all_results = {}
for verb, values in osrm_docker_params.items():
    all_results[verb] = pd.read_csv(
        f"results/all_route_list_carinal_pairs_ids/{verb}.csv"
    )
    # all_results[verb].head()

for verb, df in all_results.items():
    ic(verb)
    count_df = df.groupby("node_id", as_index=False).count()
    ic(count_df)
    ic(count_df.columns)
    # ic(all_street_nodes.columns)
    # count_df.rename(columns={"count_df.head()":"count_of_routes"}, inplace=True)
    # ic(count_df.head())
    # route_node_df =  all_street_nodes[all_street_nodes.id.isin(route_nodeids)]
    # single_gdf = all_street_nodes.join(count_df, how='inner')

    # node_count_gdf = all_street_nodes.join(count_df.set_index('node_id'))
    # count_df.rename({lambda x: x+verb}, axis='columns', inplace=True)
    count_df.rename(columns={"route_id": f"{verb}_count"}, inplace=True)
    ic(count_df.columns)
    if single_df is None:
        single_df = count_df.set_index("node_id")
    else:
        single_df = single_df.join(
            count_df.set_index("node_id"), how="outer", rsuffix=verb
        )

    # ic(len(single_df))
    # ic(single_df.columns)

# single_gdf = all_street_nodes.join(single_gdf, how='inner', lsuffix='all', rsuffix=verb)

single_df = single_df.fillna(value=0)

single_df["d_c"] = single_df.apply(
    lambda row: row["driving_count"] - row["cycling_count"], axis=1
)
single_df["d_w"] = single_df.apply(
    lambda row: row["driving_count"] - row["walking_count"], axis=1
)


ic(single_df.head())
# single_gdf.plot(column='route_id')

single_gdf = all_street_nodes.set_index("id").join(single_df, how="inner")
ic(single_gdf.head())
single_gdf.plot(column="d_c")

single_gdf.to_file("results/count_by_mode_carinal_pairs.shp")

In [ ]:
cdf = all_results["cycling"]
# cdf.head()
interesting_route_ids = interesting_routes.index.values
# # interesting_routes.
interesting_nodes = cdf.loc[interesting_route_ids].set_index("node_id")
ic(interesting_nodes.head())
ic(len(cdf))
ic(len(interesting_nodes))
ic(len(interesting_route_ids))

interesting_count_df = interesting_nodes.groupby("node_id", as_index=True).count()
interesting_count_df.rename(columns={"route_id": "cycling_count"}, inplace=True)
ic(len(interesting_count_df))
ic(interesting_count_df.columns)
interesting_count_df.info()
interesting_count_df = interesting_count_df.fillna(value=0)
ic.enable()
ic(interesting_count_df.head())
ic(all_street_nodes.head())

interesting_gdf = all_street_nodes.join(interesting_count_df, how="inner", on="id")
ic(interesting_gdf.head())
interesting_gdf.plot(column="cycling_count")
interesting_gdf.to_file("results/interesting_nodes_carinal_pairs.shp")

# Calculate the differences

# Pull it all together in a map

In [ ]:
m = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=(55.00713, -1.6202478),  # 55.0042099, -1.6202478
    zoom=16,
)
# ic(m)

# hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},

wards_geo_data = GeoData(
    geo_dataframe=aoi_wards_ll,
    style={
        "color": "black",
        "fillColor": "darkblue",
        "opacity": 0.5,
        "weight": 1.9,
        "dashArray": "2",
        "fillOpacity": 0,
    },
    name="Wards",
)
# ic(geo_data)

uprn_geo_data = GeoData(
    geo_dataframe=aoi_uprn_df,
    style={"color": "red", "opacity": 0.5, "radius": 1},
    point_style={"color": "red", "opacity": 0.5, "radius": 1},
    name="UPRN",
)

street_nodes = GeoData(
    geo_dataframe=all_street_nodes,
    style={"color": "blue", "opacity": 0.5, "radius": 1},
    point_style={"color": "blue", "opacity": 0.5, "radius": 1},
    name="Street Nodes",
)

test_route = GeoData(
    geo_dataframe=route_node_df,
    style={
        "color": "green",
        "fillColor": "green",
        "opacity": 1,
        "fillOpacity": 1,
        "radius": 3,
    },
    point_style={"color": "green", "opacity": 1, "radius": 3},
    name="test_route",
)

results = GeoData(
    geo_dataframe=single_gdf, choro_data=single_gdf, key_on="d_c", name="results"
)


single_gdf


m.add_layer(wards_geo_data)
# m.add_layer(street_nodes)
# m.add_layer(uprn_geo_data)
m.add_layer(test_route)
# m.add_layer(results)
m.add_control(LayersControl())
m